In [149]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

from tqdm import tqdm
import os
import PIL.Image as Image

import torchvision
from torch.optim.lr_scheduler import StepLR

In [29]:
from data import initialize_data, training_transforms, test_transforms # data.py in the same folder
initialize_data('images/') # extracts the zip files, makes a validation set

In [80]:
training_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.Resize((30, 30)),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
])

test_transforms = transforms.Compose([
    transforms.Resize((30, 30)),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
])

In [94]:
train_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder('images/train_images',
                         transform=training_transforms),
    batch_size=512, shuffle=True, num_workers = 4)

val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder('images/val_images',
                         transform=test_transforms),
    batch_size=512, shuffle=False, num_workers =4)

In [95]:
device = torch.device('cuda')

In [96]:
nclasses = 43 # GTSRB as 43 classes

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(500, 50)
        self.fc2 = nn.Linear(50, nclasses)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 500)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [97]:
from torchvision.models.resnet import ResNet, BasicBlock

In [139]:
class GTSRB_ResNet(ResNet):
    def __init__(self):
        super(GTSRB_ResNet, self).__init__(BasicBlock, [2, 2, 2, 2], num_classes=nclasses)
        
    def forward(self, x):
        return F.log_softmax(
            super(GTSRB_ResNet, self).forward(x), dim=-1)

In [ ]:
# model = Sequential()
# model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
# model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPool2D(pool_size=(2, 2)))
# model.add(Dropout(rate=0.25))
# model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPool2D(pool_size=(2, 2)))
# model.add(Dropout(rate=0.25))
# model.add(Flatten())
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(rate=0.5))
# model.add(Dense(43, activation='softmax'))

In [194]:
class Conv4Net(nn.Module):
    def __init__(self):
        super(Conv4Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3)
        self.fc1 = nn.Linear(64*13*13, 256)
        self.fc2 = nn.Linear(256, nclasses)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = F.dropout(x, training=self.training, p=0.25)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)
        x = F.dropout(x, training=self.training, p=0.25)
        x = x.view(-1, 64*13*13)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training, p=0.5)
        x = self.fc2(x)
        return F.log_softmax(x)

In [195]:
model = Conv4Net().to(device)

In [196]:
optimizer = torch.optim.Adam(model.parameters())

In [197]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).to(device), Variable(target).to(device)
        model.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [198]:
def validation():
    model.eval()
    validation_loss = 0
    correct = 0
    for data, target in val_loader:
        data, target = Variable(data, volatile=True).to(device), Variable(target).to(device)
        output = model(data)
        validation_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    validation_loss /= len(val_loader.dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        validation_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))

In [199]:
epochs = 25

In [200]:
exp_name = 'conv4_25_'

In [201]:
for epoch in range(1, epochs + 1):
    train(epoch)
    validation()
    if epoch%5 == 0:
        model_file = exp_name + str(epoch) + '.pth'
        torch.save(model.state_dict(), model_file)
        print('\nSaved model to ' + model_file + '. You can run `python evaluate.py --model ' + model_file + '` to generate the Kaggle formatted csv file')


/home/rm5310/pyenv/py3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/35339 (0%)]	Loss: 3.765448
Train Epoch: 1 [5120/35339 (14%)]	Loss: 3.315608
Train Epoch: 1 [10240/35339 (29%)]	Loss: 2.815134
Train Epoch: 1 [15360/35339 (43%)]	Loss: 2.210364
Train Epoch: 1 [20480/35339 (57%)]	Loss: 1.980068
Train Epoch: 1 [25600/35339 (71%)]	Loss: 1.737659
Train Epoch: 1 [30720/35339 (86%)]	Loss: 1.528002


/home/rm5310/pyenv/py3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
/home/rm5310/pyenv/py3.6.3/lib/python3.6/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Validation set: Average loss: 2.1321, Accuracy: 1543/3870 (39%)

Train Epoch: 2 [0/35339 (0%)]	Loss: 1.354611
Train Epoch: 2 [5120/35339 (14%)]	Loss: 1.263055
Train Epoch: 2 [10240/35339 (29%)]	Loss: 1.019862
Train Epoch: 2 [15360/35339 (43%)]	Loss: 1.021282
Train Epoch: 2 [20480/35339 (57%)]	Loss: 0.823481
Train Epoch: 2 [25600/35339 (71%)]	Loss: 0.793937
Train Epoch: 2 [30720/35339 (86%)]	Loss: 0.680789

Validation set: Average loss: 1.2110, Accuracy: 2512/3870 (64%)

Train Epoch: 3 [0/35339 (0%)]	Loss: 0.667673
Train Epoch: 3 [5120/35339 (14%)]	Loss: 0.594134
Train Epoch: 3 [10240/35339 (29%)]	Loss: 0.537711
Train Epoch: 3 [15360/35339 (43%)]	Loss: 0.590268
Train Epoch: 3 [20480/35339 (57%)]	Loss: 0.420641
Train Epoch: 3 [25600/35339 (71%)]	Loss: 0.472171
Train Epoch: 3 [30720/35339 (86%)]	Loss: 0.407196

Validation set: Average loss: 0.9598, Accuracy: 2849/3870 (73%)

Train Epoch: 4 [0/35339 (0%)]	Loss: 0.509261
Train Epoch: 4 [5120/35339 (14%)]	Loss: 0.528351
Train Epoch: 4 [1024

Train Epoch: 20 [30720/35339 (86%)]	Loss: 0.072787

Validation set: Average loss: 0.5770, Accuracy: 3441/3870 (88%)


Saved model to conv4_25_20.pth. You can run `python evaluate.py --model conv4_25_20.pth` to generate the Kaggle formatted csv file
Train Epoch: 21 [0/35339 (0%)]	Loss: 0.070144
Train Epoch: 21 [5120/35339 (14%)]	Loss: 0.109877
Train Epoch: 21 [10240/35339 (29%)]	Loss: 0.086206
Train Epoch: 21 [15360/35339 (43%)]	Loss: 0.072418
Train Epoch: 21 [20480/35339 (57%)]	Loss: 0.080269
Train Epoch: 21 [25600/35339 (71%)]	Loss: 0.139223
Train Epoch: 21 [30720/35339 (86%)]	Loss: 0.112620

Validation set: Average loss: 0.5555, Accuracy: 3407/3870 (88%)

Train Epoch: 22 [0/35339 (0%)]	Loss: 0.100357
Train Epoch: 22 [5120/35339 (14%)]	Loss: 0.071013
Train Epoch: 22 [10240/35339 (29%)]	Loss: 0.073010
Train Epoch: 22 [15360/35339 (43%)]	Loss: 0.038885
Train Epoch: 22 [20480/35339 (57%)]	Loss: 0.063011
Train Epoch: 22 [25600/35339 (71%)]	Loss: 0.065824
Train Epoch: 22 [30720/35339 (86%)

In [202]:
state_dict = torch.load('conv4_25_25.pth')
model.load_state_dict(state_dict)
model.eval()

Conv4Net(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=10816, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=43, bias=True)
)

In [ ]:
test_dir = 'images/test_images'

def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')

outfile = exp_name+'out.csv'
        
output_file = open(outfile, "w")
output_file.write("Filename,ClassId\n")
for f in tqdm(os.listdir(test_dir)):
    if 'ppm' in f:
        data = test_transforms(pil_loader(test_dir + '/' + f))
        data = data.view(1, data.size(0), data.size(1), data.size(2))
        data = Variable(data, volatile=True).to(device)
        output = model(data)
        pred = output.data.max(1, keepdim=True)[1]

        file_id = f[0:5]
        output_file.write("%s,%d\n" % (file_id, pred))

output_file.close()

print("Succesfully wrote " + outfile + ', you can upload this file to the kaggle '
      'competition at https://www.kaggle.com/c/nyu-cv-fall-2018/')
        

  0%|          | 0/12630 [00:00<?, ?it/s]/home/rm5310/pyenv/py3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/rm5310/pyenv/py3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
 28%|██▊       | 3497/12630 [00:50<02:10, 69.89it/s]